In [50]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [51]:
ail_df = pd.read_csv('../../data/raw/ail_price.csv', parse_dates=['Date (MST)'], index_col='Date (MST)')
ail_df = ail_df.rename_axis('date')
ail_df = ail_df.asfreq('H')
ail_df = ail_df.sort_values(by='date')
ail_df.head()

,Date - MST,Hourly Profile1,Season1,AIL,Gas Price,Price,Spark Spread
date,,,,,,,
2021-01-01 00:00:00,1/1/2021 12:00:00 AM,OFF PEAK,WINTER,9655,2.4633,29.92,11.44525
2021-01-01 01:00:00,1/1/2021 1:00:00 AM,OFF PEAK,WINTER,9513,2.4633,27.48,9.00525
2021-01-01 02:00:00,1/1/2021 2:00:00 AM,OFF PEAK,WINTER,9437,2.4633,28.62,10.14525
2021-01-01 03:00:00,1/1/2021 3:00:00 AM,OFF PEAK,WINTER,9376,2.4633,33.55,15.07525
2021-01-01 04:00:00,1/1/2021 4:00:00 AM,OFF PEAK,WINTER,9356,2.4633,35.36,16.88525


In [52]:
supply_df_old = pd.read_csv('../../data/raw/gen_old.csv', parse_dates=['Date (MST)'], index_col='Date (MST)')
supply_df_old = supply_df_old.sort_values(by='Date (MST)')
supply_df_old

earliest_datetime = supply_df_old.index.min()
latest_datetime = supply_df_old.index.max()

print("Earliest datetime:", earliest_datetime)
print("Latest datetime:", latest_datetime)

Earliest datetime: 2020-12-27 00:00:00
Latest datetime: 2023-04-30 22:00:00


In [53]:
supply_df = pd.read_csv('../../data/raw/gen.csv', parse_dates=['Date (MST)'], index_col='Date (MST)')
supply_df = supply_df.sort_values(by='Date (MST)')
supply_df.head(2)

,Availability Factor,Availability Utilization,Capacity Factor,Date,Date - MST,Date (MPT),Fuel Type,Hourly Profile,Maximum Capacity,Season,System Available,System Capacity,System Generation,Total Generation
Date (MST),,,,,,,,,,,,,,
2020-12-12,0.917021,0.564442,0.521384,12/12/2020,12/12/2020 12:00:00 AM,12/12/2020 12:00:00 AM,Dual Fuel,OFF PEAK,940,WINTER,862,940.0,486.548579,490.100850
2020-12-12,0.430446,0.450396,0.683408,12/12/2020,12/12/2020 12:00:00 AM,12/12/2020 12:00:00 AM,Other,OFF PEAK,381,WINTER,164,270.9,73.864900,260.378488


### Checking if the new data is good enough

In [54]:
# Convert the index to datetime if it's not already
supply_df_old.index = pd.to_datetime(supply_df_old.index)

start_date = pd.to_datetime('2021-10-01 00:00:00')
end_date = pd.to_datetime('2023-04-06 23:59:59')

# Perform the query to retrieve data between the specified dates
query_result = supply_df_old.loc[(supply_df_old.index >= start_date) & (supply_df_old.index <= end_date)]
query_result.shape


(140209, 12)

In [55]:
# Convert the index to datetime if it's not already
supply_df.index = pd.to_datetime(supply_df.index)

start_date = pd.to_datetime('2021-10-01 00:00:00')
end_date = pd.to_datetime('2023-04-06 23:59:59')

# Perform the query to retrieve data between the specified dates
query_result = supply_df.loc[(supply_df.index >= start_date) & (supply_df.index <= end_date)]
query_result.shape


(140209, 14)

We have the same number of rows in the date range which means that the data is good enough. Proceed with the next steps

In [56]:
reset_df = supply_df.reset_index()
reset_df

gen_df  = reset_df[['Date (MST)', 'Fuel Type', 'Total Generation']]
wide_tng_df = gen_df.pivot(index='Date (MST)', columns='Fuel Type', values='Total Generation')

sys_df  = reset_df[['Date (MST)', 'Fuel Type', 'System Generation']]
wide_sys_df = sys_df.pivot(index='Date (MST)', columns='Fuel Type', values='System Generation')

sys_cap_df  = reset_df[['Date (MST)', 'Fuel Type', 'System Capacity']]
wide_sys_cap_df = sys_cap_df.pivot(index='Date (MST)', columns='Fuel Type', values='System Capacity')

sys_avail_df  = reset_df[['Date (MST)', 'Fuel Type', 'System Available']]
wide_sys_avail_df = sys_avail_df.pivot(index='Date (MST)', columns='Fuel Type', values='System Available')

max_cap_df  = reset_df[['Date (MST)', 'Fuel Type', 'Maximum Capacity']]
wide_max_cap_df = max_cap_df.pivot(index='Date (MST)', columns='Fuel Type', values='Maximum Capacity')

cap_fac_df  = reset_df[['Date (MST)', 'Fuel Type', 'Capacity Factor']]
wide_cap_fac_df = cap_fac_df.pivot(index='Date (MST)', columns='Fuel Type', values='Capacity Factor')

avail_util  = reset_df[['Date (MST)', 'Fuel Type', 'Availability Utilization']]
wide_avail_util = avail_util.pivot(index='Date (MST)', columns='Fuel Type', values='Availability Utilization')

avail_fact  = reset_df[['Date (MST)', 'Fuel Type', 'Availability Factor']]
wide_avail_fact = avail_fact.pivot(index='Date (MST)', columns='Fuel Type', values='Availability Factor')

In [57]:
wide_tng_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 21623 entries, 2020-12-12 00:00:00 to 2023-05-31 22:00:00
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Coal             21623 non-null  float64
 1   Cogeneration     21623 non-null  float64
 2   Combined Cycle   21623 non-null  float64
 3   Dual Fuel        15840 non-null  float64
 4   Gas Fired Steam  20399 non-null  float64
 5   Hydro            21623 non-null  float64
 6   Other            21623 non-null  float64
 7   Simple Cycle     21623 non-null  float64
 8   Solar            21623 non-null  float64
 9   Storage          21623 non-null  float64
 10  Wind             21623 non-null  float64
dtypes: float64(11)
memory usage: 2.0 MB


In [58]:
wide_tng_df = wide_tng_df.fillna(0)
wide_sys_avail_df = wide_sys_avail_df.fillna(0)

In [59]:
column_mapping = {
    'Gas Fired Steam': 'gas_fired_steam_tng',
    'Combined Cycle': 'combined_cycle_tng',
    'Simple Cycle': 'simple_cycle_tng',
    'Solar': 'solar_tng',
    'Storage': 'storage_tng',
    'Wind': 'wind_tng',
    'Hydro': 'hydro_tng',
    'Cogeneration': 'cogeneration_tng',
    'Coal': 'coal_tng',
    'Dual Fuel': 'dual_fuel_tng',
    'Other': 'other_tng',
}

# Rename the columns using the mapping
wide_tng_df = wide_tng_df.rename(columns=column_mapping)
wide_tng_df

Fuel Type,coal_tng,cogeneration_tng,combined_cycle_tng,dual_fuel_tng,gas_fired_steam_tng,hydro_tng,other_tng,simple_cycle_tng,solar_tng,storage_tng,wind_tng
Date (MST),,,,,,,,,,,
2020-12-12 00:00:00,2391.710560,4197.060646,1449.084680,490.100850,0.000000,85.275470,260.378488,380.394305,0.000000,0.000000,255.670821
2020-12-12 01:00:00,2426.215458,4188.542116,1427.123231,337.071535,0.000000,71.033118,260.995378,378.669994,0.000000,0.000000,263.779892
2020-12-12 02:00:00,2330.198275,4177.570336,1442.588052,331.934240,0.000000,80.803783,260.409256,378.042595,0.000000,0.000000,269.793849
2020-12-12 03:00:00,2329.907512,4168.678630,1448.433288,303.827912,0.000000,83.682327,259.855309,375.117403,0.000000,0.000000,250.563136
2020-12-12 04:00:00,2330.164927,4178.184740,1439.484858,339.096623,0.000000,68.867599,257.689517,375.659958,0.000000,0.000000,247.508450
...,...,...,...,...,...,...,...,...,...,...,...
2023-05-31 18:00:00,802.061262,3183.402464,1494.712151,465.082272,1129.959460,344.680324,272.632742,455.394841,207.039517,0.026556,939.408382
2023-05-31 19:00:00,802.619933,3184.047986,1482.522364,465.009882,1032.034942,312.848796,267.052824,429.854741,74.041111,0.004056,989.357987
2023-05-31 20:00:00,801.494044,3168.761255,1430.846669,464.984549,1079.687323,287.204005,247.692334,422.858229,4.252167,0.000833,872.252299


In [60]:
wide_tng_df['gas_tng'] = wide_tng_df['cogeneration_tng'] + wide_tng_df['combined_cycle_tng'] + wide_tng_df['gas_fired_steam_tng'] + wide_tng_df['simple_cycle_tng']

In [61]:
tng_df = wide_tng_df[['gas_tng', 'dual_fuel_tng', 'coal_tng', 'wind_tng', 'solar_tng', 'hydro_tng', 'storage_tng', 'other_tng']]

In [62]:
tng_df.columns = ['gas_tng', 'dual_fuel_tng', 'coal_tng', 'wind_tng', 'solar_tng', 'hydro_tng', 'storage_tng', 'other_tng']

In [63]:
tng_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 21623 entries, 2020-12-12 00:00:00 to 2023-05-31 22:00:00
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gas_tng        21623 non-null  float64
 1   dual_fuel_tng  21623 non-null  float64
 2   coal_tng       21623 non-null  float64
 3   wind_tng       21623 non-null  float64
 4   solar_tng      21623 non-null  float64
 5   hydro_tng      21623 non-null  float64
 6   storage_tng    21623 non-null  float64
 7   other_tng      21623 non-null  float64
dtypes: float64(8)
memory usage: 1.5 MB


In [64]:
column_mapping = {
    'Gas Fired Steam': 'gas_fired_steam_avail',
    'Combined Cycle': 'combined_cycle_avail',
    'Simple Cycle': 'simple_cycle_avail',
    'Solar': 'solar_avail',
    'Storage': 'storage_avail',
    'Wind': 'wind_avail',
    'Hydro': 'hydro_avail',
    'Cogeneration': 'cogeneration_avail',
    'Coal': 'coal_avail',
    'Dual Fuel': 'dual_fuel_avail',
    'Other': 'other_avail',
}

wide_sys_avail_df = wide_sys_avail_df.rename(columns=column_mapping)
wide_sys_avail_df.head()

Fuel Type,coal_avail,cogeneration_avail,combined_cycle_avail,dual_fuel_avail,gas_fired_steam_avail,hydro_avail,other_avail,simple_cycle_avail,solar_avail,storage_avail,wind_avail
Date (MST),,,,,,,,,,,
2020-12-12 00:00:00,2776.0,3359.0,1647.0,862.0,0.0,709.0,164.0,938.0,107.0,10.0,1781.0
2020-12-12 01:00:00,2862.0,3359.0,1650.0,862.0,0.0,709.0,164.0,938.0,107.0,10.0,1781.0
2020-12-12 02:00:00,2862.0,3349.0,1675.0,862.0,0.0,709.0,164.0,938.0,107.0,10.0,1781.0
2020-12-12 03:00:00,2862.0,3349.0,1679.0,862.0,0.0,709.0,164.0,940.0,107.0,10.0,1781.0
2020-12-12 04:00:00,2862.0,3369.0,1679.0,862.0,0.0,709.0,164.0,941.0,107.0,10.0,1781.0


In [65]:
wide_sys_avail_df['gas_avail'] = wide_sys_avail_df['cogeneration_avail'] + wide_sys_avail_df['combined_cycle_avail'] + wide_sys_avail_df['gas_fired_steam_avail'] + wide_sys_avail_df['simple_cycle_avail']

In [66]:
avail_df = wide_sys_avail_df[['gas_avail', 'dual_fuel_avail', 'coal_avail', 'wind_avail', 'solar_avail', 'hydro_avail', 'storage_avail', 'other_avail']]

In [67]:
avail_df.columns = ['gas_avail', 'dual_fuel_avail', 'coal_avail', 'wind_avail', 'solar_avail', 'hydro_avail', 'storage_avail', 'other_avail']

In [68]:
avail_df.head()

,gas_avail,dual_fuel_avail,coal_avail,wind_avail,solar_avail,hydro_avail,storage_avail,other_avail
Date (MST),,,,,,,,
2020-12-12 00:00:00,5944.0,862.0,2776.0,1781.0,107.0,709.0,10.0,164.0
2020-12-12 01:00:00,5947.0,862.0,2862.0,1781.0,107.0,709.0,10.0,164.0
2020-12-12 02:00:00,5962.0,862.0,2862.0,1781.0,107.0,709.0,10.0,164.0
2020-12-12 03:00:00,5968.0,862.0,2862.0,1781.0,107.0,709.0,10.0,164.0
2020-12-12 04:00:00,5989.0,862.0,2862.0,1781.0,107.0,709.0,10.0,164.0


In [69]:
merged_df = pd.concat([tng_df, avail_df], axis=1)
merged_df.head()

,gas_tng,dual_fuel_tng,coal_tng,wind_tng,solar_tng,hydro_tng,storage_tng,other_tng,gas_avail,dual_fuel_avail,coal_avail,wind_avail,solar_avail,hydro_avail,storage_avail,other_avail
Date (MST),,,,,,,,,,,,,,,,
2020-12-12 00:00:00,6026.539631,490.100850,2391.710560,255.670821,0.0,85.275470,0.0,260.378488,5944.0,862.0,2776.0,1781.0,107.0,709.0,10.0,164.0
2020-12-12 01:00:00,5994.335342,337.071535,2426.215458,263.779892,0.0,71.033118,0.0,260.995378,5947.0,862.0,2862.0,1781.0,107.0,709.0,10.0,164.0
2020-12-12 02:00:00,5998.200984,331.934240,2330.198275,269.793849,0.0,80.803783,0.0,260.409256,5962.0,862.0,2862.0,1781.0,107.0,709.0,10.0,164.0
2020-12-12 03:00:00,5992.229321,303.827912,2329.907512,250.563136,0.0,83.682327,0.0,259.855309,5968.0,862.0,2862.0,1781.0,107.0,709.0,10.0,164.0
2020-12-12 04:00:00,5993.329555,339.096623,2330.164927,247.508450,0.0,68.867599,0.0,257.689517,5989.0,862.0,2862.0,1781.0,107.0,709.0,10.0,164.0


In [70]:
import numpy as np

fuel_types = ['gas', 'coal', 'wind', 'solar', 'hydro', 'storage', 'other']

for fuel_type in fuel_types:
    tng_column = f'{fuel_type}_tng'
    avail_column = f'{fuel_type}_avail'
    reserve_margin_column = f'{fuel_type}_reserve_margin'
    merged_df[reserve_margin_column] = np.where(merged_df[avail_column] == 0, 0, 1 - (merged_df[tng_column] / merged_df[avail_column]))

In [71]:
merged_df.head()

,gas_tng,dual_fuel_tng,coal_tng,wind_tng,solar_tng,hydro_tng,storage_tng,other_tng,gas_avail,dual_fuel_avail,...,hydro_avail,storage_avail,other_avail,gas_reserve_margin,coal_reserve_margin,wind_reserve_margin,solar_reserve_margin,hydro_reserve_margin,storage_reserve_margin,other_reserve_margin
Date (MST),,,,,,,,,,,,,,,,,,,,,
2020-12-12 00:00:00,6026.539631,490.100850,2391.710560,255.670821,0.0,85.275470,0.0,260.378488,5944.0,862.0,...,709.0,10.0,164.0,-0.013886,0.138433,0.856445,1.0,0.879724,1.0,-0.587674
2020-12-12 01:00:00,5994.335342,337.071535,2426.215458,263.779892,0.0,71.033118,0.0,260.995378,5947.0,862.0,...,709.0,10.0,164.0,-0.007960,0.152266,0.851892,1.0,0.899812,1.0,-0.591435
2020-12-12 02:00:00,5998.200984,331.934240,2330.198275,269.793849,0.0,80.803783,0.0,260.409256,5962.0,862.0,...,709.0,10.0,164.0,-0.006072,0.185815,0.848516,1.0,0.886031,1.0,-0.587861
2020-12-12 03:00:00,5992.229321,303.827912,2329.907512,250.563136,0.0,83.682327,0.0,259.855309,5968.0,862.0,...,709.0,10.0,164.0,-0.004060,0.185916,0.859313,1.0,0.881971,1.0,-0.584484
2020-12-12 04:00:00,5993.329555,339.096623,2330.164927,247.508450,0.0,68.867599,0.0,257.689517,5989.0,862.0,...,709.0,10.0,164.0,-0.000723,0.185826,0.861028,1.0,0.902867,1.0,-0.571278


In [72]:
column_mapping = {
    'Gas Price': 'gas_price',
    'Price': 'price',
    'Hourly Profile1': 'peak_or_not',
    'Season1': 'season',
    'AIL': 'ail',
}

ail_df = ail_df.rename(columns=column_mapping)
ail_df

,Date - MST,peak_or_not,season,ail,gas_price,price,Spark Spread
date,,,,,,,
2021-01-01 00:00:00,1/1/2021 12:00:00 AM,OFF PEAK,WINTER,9655,2.4633,29.92,11.44525
2021-01-01 01:00:00,1/1/2021 1:00:00 AM,OFF PEAK,WINTER,9513,2.4633,27.48,9.00525
2021-01-01 02:00:00,1/1/2021 2:00:00 AM,OFF PEAK,WINTER,9437,2.4633,28.62,10.14525
2021-01-01 03:00:00,1/1/2021 3:00:00 AM,OFF PEAK,WINTER,9376,2.4633,33.55,15.07525
2021-01-01 04:00:00,1/1/2021 4:00:00 AM,OFF PEAK,WINTER,9356,2.4633,35.36,16.88525
...,...,...,...,...,...,...,...
2023-05-31 18:00:00,5/31/2023 6:00:00 PM,ON PEAK,SUMMER,9566,2.0700,188.54,173.01500
2023-05-31 19:00:00,5/31/2023 7:00:00 PM,ON PEAK,SUMMER,9383,2.0700,104.75,89.22500
2023-05-31 20:00:00,5/31/2023 8:00:00 PM,ON PEAK,SUMMER,9141,2.0700,53.52,37.99500


In [73]:
ail_df = ail_df[['ail', 'gas_price', 'price', 'peak_or_not', 'season']]

In [74]:
final_df = pd.merge(ail_df, merged_df,  left_index=True, right_on='Date (MST)')

In [75]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 21143 entries, 2021-01-01 00:00:00 to 2023-05-31 22:00:00
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ail                     21143 non-null  int64  
 1   gas_price               21143 non-null  float64
 2   price                   21143 non-null  float64
 3   peak_or_not             21143 non-null  object 
 4   season                  21143 non-null  object 
 5   gas_tng                 21143 non-null  float64
 6   dual_fuel_tng           21143 non-null  float64
 7   coal_tng                21143 non-null  float64
 8   wind_tng                21143 non-null  float64
 9   solar_tng               21143 non-null  float64
 10  hydro_tng               21143 non-null  float64
 11  storage_tng             21143 non-null  float64
 12  other_tng               21143 non-null  float64
 13  gas_avail               21143 non-null  float64
 14  dua

In [76]:
final_df['total_tng'] = final_df['gas_tng'] + final_df['dual_fuel_tng'] + final_df['coal_tng'] + final_df['wind_tng'] + \
                        final_df['solar_avail'] + final_df['hydro_tng'] + final_df['storage_tng'] + \
                        final_df['other_tng']

In [77]:
final_df['total_avail'] = final_df['gas_avail'] + final_df['dual_fuel_avail'] + final_df['coal_avail'] + final_df['wind_avail'] + \
                        final_df['solar_avail'] + final_df['hydro_avail'] + final_df['storage_avail'] + \
                        final_df['other_avail']

In [78]:
import numpy as np
fuel_types = ['gas', 'dual_fuel', 'coal', 'wind', 'solar', 'hydro', 'storage', 'other']

for fuel_type in fuel_types:
    tng_column = f'{fuel_type}_tng'
    supply_mix_column = f'{fuel_type}_supply_mix'
    final_df[supply_mix_column] = (final_df[tng_column] / final_df['total_tng'])

In [79]:
final_df['total_reserve_margin'] = 1 - (final_df['total_tng']/final_df['total_avail'])

In [80]:
final_df['relative_gas_reserve'] =  final_df['gas_reserve_margin'] / final_df['total_reserve_margin']

In [81]:
final_df['demand_supply_ratio'] =  final_df['ail'] / final_df['total_avail']
final_df['avail_gen_ratio'] = final_df['total_avail'] / final_df['total_tng']

In [82]:
final_df['load_on_gas_reserve'] = (final_df['gas_avail'] - final_df['gas_tng']) / (final_df['ail'] * final_df['gas_supply_mix'])

In [83]:
final_df['renewable_energy_ratio'] = (final_df['wind_tng'] + final_df['solar_tng'] + final_df['hydro_tng'] + final_df['storage_tng']) / final_df['total_tng']

final_df['fossil_fuel_ratio'] = (final_df['gas_tng'] + final_df['coal_tng']) / final_df['total_tng']

final_df['energy_reserve_margin'] = final_df['total_avail'] - final_df['total_tng']

final_df['gas_cost'] = (final_df['gas_supply_mix'] * final_df['ail'] * final_df['gas_price']) 

final_df['gas_avail_ratio'] = final_df['gas_avail'] / final_df['total_avail']
final_df['coal_avail_ratio'] = final_df['coal_avail'] / final_df['total_avail']

final_df['gas_tng_ratio'] = final_df['gas_tng'] / final_df['total_tng']
final_df['coal_tng_ratio'] = final_df['coal_tng'] / final_df['total_tng']

final_df['renewable_energy_penetration'] = (final_df['wind_supply_mix'] + final_df['solar_supply_mix'] + final_df['hydro_supply_mix'] + final_df['storage_supply_mix']) / 100


In [84]:
# Compute the correlation matrix
corr_matrix = final_df.corr('spearman')

# Filter correlations with magnitude greater than 0.5
filtered_corr_matrix = corr_matrix[abs(corr_matrix['price']) > 0.5]

# Select only the 'price' column and the correlated features
price_corr = filtered_corr_matrix[['price']]

price_corr = price_corr.iloc[1:]
# Apply background gradient to the correlation matrix
styled_corr_matrix = price_corr.style.background_gradient()

# Display the styled correlation matrix
styled_corr_matrix

,price
gas_tng,0.534247
wind_reserve_margin,0.501942
gas_supply_mix,0.547282
gas_tng_ratio,0.547282


In [85]:

# final_df = final_df[final_df.index.year >= 2022]
final_df.head(2)

,ail,gas_price,price,peak_or_not,season,gas_tng,dual_fuel_tng,coal_tng,wind_tng,solar_tng,...,load_on_gas_reserve,renewable_energy_ratio,fossil_fuel_ratio,energy_reserve_margin,gas_cost,gas_avail_ratio,coal_avail_ratio,gas_tng_ratio,coal_tng_ratio,renewable_energy_penetration
Date (MST),,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,9655,2.4633,29.92,OFF PEAK,WINTER,5699.574142,134.160065,1838.142405,1470.686241,0.0,...,0.047723,0.159624,0.787665,3401.306600,14164.914865,0.460566,0.253026,0.595586,0.192080,0.001596
2021-01-01 01:00:00,9513,2.4633,27.48,OFF PEAK,WINTER,5679.769352,132.957960,1669.156269,1525.467843,0.0,...,0.052878,0.169058,0.777388,3130.645824,14079.251738,0.475366,0.229418,0.600821,0.176568,0.001691


In [86]:
final_df.tail(2)

,ail,gas_price,price,peak_or_not,season,gas_tng,dual_fuel_tng,coal_tng,wind_tng,solar_tng,...,load_on_gas_reserve,renewable_energy_ratio,fossil_fuel_ratio,energy_reserve_margin,gas_cost,gas_avail_ratio,coal_avail_ratio,gas_tng_ratio,coal_tng_ratio,renewable_energy_penetration
Date (MST),,,,,,,,,,,,,,,,,,,,,
2023-05-31 21:00:00,9007,2.07,49.71,ON PEAK,SUMMER,6030.283728,465.019773,803.622156,895.844240,0.0,...,0.145486,0.119651,0.700268,3873.011486,11520.821496,0.501761,0.059052,0.617921,0.082347,0.001197
2023-05-31 22:00:00,8692,2.07,55.85,OFF PEAK,SUMMER,5726.632803,464.987881,805.713989,877.518179,0.0,...,0.220400,0.121826,0.692121,4242.838106,10916.966488,0.503545,0.058841,0.606753,0.085368,0.001218


In [87]:
final_df.info()
final_df

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 21143 entries, 2021-01-01 00:00:00 to 2023-05-31 22:00:00
Data columns (total 52 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ail                           21143 non-null  int64  
 1   gas_price                     21143 non-null  float64
 2   price                         21143 non-null  float64
 3   peak_or_not                   21143 non-null  object 
 4   season                        21143 non-null  object 
 5   gas_tng                       21143 non-null  float64
 6   dual_fuel_tng                 21143 non-null  float64
 7   coal_tng                      21143 non-null  float64
 8   wind_tng                      21143 non-null  float64
 9   solar_tng                     21143 non-null  float64
 10  hydro_tng                     21143 non-null  float64
 11  storage_tng                   21143 non-null  float64
 12  other_tng                

,ail,gas_price,price,peak_or_not,season,gas_tng,dual_fuel_tng,coal_tng,wind_tng,solar_tng,...,load_on_gas_reserve,renewable_energy_ratio,fossil_fuel_ratio,energy_reserve_margin,gas_cost,gas_avail_ratio,coal_avail_ratio,gas_tng_ratio,coal_tng_ratio,renewable_energy_penetration
Date (MST),,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,9655,2.4633,29.92,OFF PEAK,WINTER,5699.574142,134.160065,1838.142405,1470.686241,0.000000,...,0.047723,0.159624,0.787665,3401.306600,14164.914865,0.460566,0.253026,0.595586,0.192080,0.001596
2021-01-01 01:00:00,9513,2.4633,27.48,OFF PEAK,WINTER,5679.769352,132.957960,1669.156269,1525.467843,0.000000,...,0.052878,0.169058,0.777388,3130.645824,14079.251738,0.475366,0.229418,0.600821,0.176568,0.001691
2021-01-01 02:00:00,9437,2.4633,28.62,OFF PEAK,WINTER,5677.664967,133.070086,1511.079347,1535.146498,0.000000,...,0.051222,0.174815,0.770960,3519.591174,14154.668958,0.464964,0.245796,0.608903,0.162056,0.001748
2021-01-01 03:00:00,9376,2.4633,33.55,OFF PEAK,WINTER,5769.198693,132.317335,1405.418798,1484.051400,0.000000,...,0.034369,0.170573,0.774941,3466.726794,14391.975452,0.469155,0.238900,0.623140,0.151801,0.001706
2021-01-01 04:00:00,9356,2.4633,35.36,OFF PEAK,WINTER,5777.915298,132.603031,1478.989098,1446.955595,0.000000,...,0.032638,0.163419,0.781837,3463.134162,14346.415484,0.468262,0.240251,0.622495,0.159342,0.001634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-31 18:00:00,9566,2.0700,188.54,ON PEAK,SUMMER,6263.468916,465.082272,802.061262,939.408382,207.039517,...,0.111884,0.145815,0.690913,3573.639546,12128.149786,0.501377,0.058333,0.612483,0.078431,0.001458
2023-05-31 19:00:00,9383,2.0700,104.75,ON PEAK,SUMMER,6128.460033,465.009882,802.619933,989.357987,74.041111,...,0.140493,0.136204,0.685950,3704.646489,11780.260329,0.501702,0.058295,0.606517,0.079433,0.001362
2023-05-31 20:00:00,9141,2.0700,53.52,ON PEAK,SUMMER,6102.153476,464.984549,801.494044,872.252299,4.252167,...,0.151415,0.117371,0.696299,3920.218460,11645.657983,0.502638,0.058186,0.615460,0.080838,0.001174


In [88]:
final_df = final_df.sort_index()
final_df = final_df.asfreq("H")
final_df

,ail,gas_price,price,peak_or_not,season,gas_tng,dual_fuel_tng,coal_tng,wind_tng,solar_tng,...,load_on_gas_reserve,renewable_energy_ratio,fossil_fuel_ratio,energy_reserve_margin,gas_cost,gas_avail_ratio,coal_avail_ratio,gas_tng_ratio,coal_tng_ratio,renewable_energy_penetration
Date (MST),,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,9655,2.4633,29.92,OFF PEAK,WINTER,5699.574142,134.160065,1838.142405,1470.686241,0.000000,...,0.047723,0.159624,0.787665,3401.306600,14164.914865,0.460566,0.253026,0.595586,0.192080,0.001596
2021-01-01 01:00:00,9513,2.4633,27.48,OFF PEAK,WINTER,5679.769352,132.957960,1669.156269,1525.467843,0.000000,...,0.052878,0.169058,0.777388,3130.645824,14079.251738,0.475366,0.229418,0.600821,0.176568,0.001691
2021-01-01 02:00:00,9437,2.4633,28.62,OFF PEAK,WINTER,5677.664967,133.070086,1511.079347,1535.146498,0.000000,...,0.051222,0.174815,0.770960,3519.591174,14154.668958,0.464964,0.245796,0.608903,0.162056,0.001748
2021-01-01 03:00:00,9376,2.4633,33.55,OFF PEAK,WINTER,5769.198693,132.317335,1405.418798,1484.051400,0.000000,...,0.034369,0.170573,0.774941,3466.726794,14391.975452,0.469155,0.238900,0.623140,0.151801,0.001706
2021-01-01 04:00:00,9356,2.4633,35.36,OFF PEAK,WINTER,5777.915298,132.603031,1478.989098,1446.955595,0.000000,...,0.032638,0.163419,0.781837,3463.134162,14346.415484,0.468262,0.240251,0.622495,0.159342,0.001634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-31 18:00:00,9566,2.0700,188.54,ON PEAK,SUMMER,6263.468916,465.082272,802.061262,939.408382,207.039517,...,0.111884,0.145815,0.690913,3573.639546,12128.149786,0.501377,0.058333,0.612483,0.078431,0.001458
2023-05-31 19:00:00,9383,2.0700,104.75,ON PEAK,SUMMER,6128.460033,465.009882,802.619933,989.357987,74.041111,...,0.140493,0.136204,0.685950,3704.646489,11780.260329,0.501702,0.058295,0.606517,0.079433,0.001362
2023-05-31 20:00:00,9141,2.0700,53.52,ON PEAK,SUMMER,6102.153476,464.984549,801.494044,872.252299,4.252167,...,0.151415,0.117371,0.696299,3920.218460,11645.657983,0.502638,0.058186,0.615460,0.080838,0.001174


In [89]:
import pandas as pd

# Assuming your DataFrame is named 'df' and the datetime index column is named 'datetime_index'
duplicated_values = final_df.index.duplicated()

if any(duplicated_values):
    print("There are repeated values in the datetime index column.")
else:
    print("There are no repeated values in the datetime index column.")


There are no repeated values in the datetime index column.


In [90]:
final_df.shape

(21143, 52)

### Region wise loads

In [91]:
region_df = pd.read_csv('../../data/raw/region_load.csv', parse_dates=['Date (MST)'], index_col='Date (MST)')
region_df = region_df.rename_axis('date')
region_df = region_df.sort_values(by='date')
region_df = region_df.asfreq("H")
region_df.tail()

columns_to_drop = ['Date - MST', 'Date', 'Hourly Profile', 'Season', 'Date (MPT)']
region_df = region_df.drop(columns=columns_to_drop)


region_df.columns = region_df.columns.str.lower().str.replace(' ', '_')
region_df.head()

,calgary,central,edmonton,losses,northeast,northwest,south,system_load
date,,,,,,,,
2020-12-28 00:00:00,973.960889,1236.320688,1387.089683,322.690556,1193.081806,815.039186,819.507700,6747.690508
2020-12-28 01:00:00,933.771618,1225.190554,1333.092374,346.431205,1213.420478,835.659613,808.163113,6695.728955
2020-12-28 02:00:00,908.266409,1219.458539,1319.365519,331.498849,1220.855952,836.783731,802.048526,6638.277525
2020-12-28 03:00:00,898.755269,1232.519591,1313.050226,329.917951,1211.050050,844.572036,799.559222,6629.424345
2020-12-28 04:00:00,909.831405,1226.592202,1322.738666,358.833736,1193.917142,861.557468,805.235644,6678.706264


In [92]:
final_region_df = pd.merge(region_df, final_df,  left_index=True, right_index=True)
final_region_df.index.name = 'date'
final_region_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 21143 entries, 2021-01-01 00:00:00 to 2023-05-31 22:00:00
Freq: H
Data columns (total 60 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   calgary                       21143 non-null  float64
 1   central                       21143 non-null  float64
 2   edmonton                      21143 non-null  float64
 3   losses                        21143 non-null  float64
 4   northeast                     21143 non-null  float64
 5   northwest                     21143 non-null  float64
 6   south                         21143 non-null  float64
 7   system_load                   21143 non-null  float64
 8   ail                           21143 non-null  int64  
 9   gas_price                     21143 non-null  float64
 10  price                         21143 non-null  float64
 11  peak_or_not                   21143 non-null  object 
 12  season           

In [93]:
final_region_df.columns

Index(['calgary', 'central', 'edmonton', 'losses', 'northeast', 'northwest',
       'south', 'system_load', 'ail', 'gas_price', 'price', 'peak_or_not',
       'season', 'gas_tng', 'dual_fuel_tng', 'coal_tng', 'wind_tng',
       'solar_tng', 'hydro_tng', 'storage_tng', 'other_tng', 'gas_avail',
       'dual_fuel_avail', 'coal_avail', 'wind_avail', 'solar_avail',
       'hydro_avail', 'storage_avail', 'other_avail', 'gas_reserve_margin',
       'coal_reserve_margin', 'wind_reserve_margin', 'solar_reserve_margin',
       'hydro_reserve_margin', 'storage_reserve_margin',
       'other_reserve_margin', 'total_tng', 'total_avail', 'gas_supply_mix',
       'dual_fuel_supply_mix', 'coal_supply_mix', 'wind_supply_mix',
       'solar_supply_mix', 'hydro_supply_mix', 'storage_supply_mix',
       'other_supply_mix', 'total_reserve_margin', 'relative_gas_reserve',
       'demand_supply_ratio', 'avail_gen_ratio', 'load_on_gas_reserve',
       'renewable_energy_ratio', 'fossil_fuel_ratio', 'energy_re

In [94]:
final_region_df.to_csv('../../data/processed/supply_load_price.csv')

In [95]:
final_region_df[final_region_df.isna().any(axis=1)]

,calgary,central,edmonton,losses,northeast,northwest,south,system_load,ail,gas_price,...,load_on_gas_reserve,renewable_energy_ratio,fossil_fuel_ratio,energy_reserve_margin,gas_cost,gas_avail_ratio,coal_avail_ratio,gas_tng_ratio,coal_tng_ratio,renewable_energy_penetration
date,,,,,,,,,,,,,,,,,,,,,


In [96]:
final_region_df

,calgary,central,edmonton,losses,northeast,northwest,south,system_load,ail,gas_price,...,load_on_gas_reserve,renewable_energy_ratio,fossil_fuel_ratio,energy_reserve_margin,gas_cost,gas_avail_ratio,coal_avail_ratio,gas_tng_ratio,coal_tng_ratio,renewable_energy_penetration
date,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,980.722146,1171.473099,1358.532734,236.922400,1174.421179,917.347644,820.224659,6659.643861,9655,2.4633,...,0.047723,0.159624,0.787665,3401.306600,14164.914865,0.460566,0.253026,0.595586,0.192080,0.001596
2021-01-01 01:00:00,930.446202,1176.854575,1319.084992,232.749707,1169.068763,921.311098,798.816485,6548.331823,9513,2.4633,...,0.052878,0.169058,0.777388,3130.645824,14079.251738,0.475366,0.229418,0.600821,0.176568,0.001691
2021-01-01 02:00:00,892.964253,1185.751442,1308.038703,230.711068,1153.448009,914.739715,789.838124,6475.491314,9437,2.4633,...,0.051222,0.174815,0.770960,3519.591174,14154.668958,0.464964,0.245796,0.608903,0.162056,0.001748
2021-01-01 03:00:00,874.536088,1176.828153,1292.884711,219.980539,1142.044135,868.807503,783.263951,6358.345079,9376,2.4633,...,0.034369,0.170573,0.774941,3466.726794,14391.975452,0.469155,0.238900,0.623140,0.151801,0.001706
2021-01-01 04:00:00,873.012668,1177.172785,1287.351410,255.139984,1139.851965,843.089582,785.215591,6360.833986,9356,2.4633,...,0.032638,0.163419,0.781837,3463.134162,14346.415484,0.468262,0.240251,0.622495,0.159342,0.001634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-31 18:00:00,1379.680924,1248.687412,1648.561087,226.832123,1023.412434,585.970201,881.953732,6995.097912,9566,2.0700,...,0.111884,0.145815,0.690913,3573.639546,12128.149786,0.501377,0.058333,0.612483,0.078431,0.001458
2023-05-31 19:00:00,1346.755145,1213.881244,1590.966760,172.560250,1026.354757,597.763107,922.811833,6871.093096,9383,2.0700,...,0.140493,0.136204,0.685950,3704.646489,11780.260329,0.501702,0.058295,0.606517,0.079433,0.001362
2023-05-31 20:00:00,1295.758512,1158.126007,1525.122006,150.578999,1034.968967,611.110713,924.559523,6700.224727,9141,2.0700,...,0.151415,0.117371,0.696299,3920.218460,11645.657983,0.502638,0.058186,0.615460,0.080838,0.001174
